In [1]:
import pandas as pd
import sys
import os
sys.path.append("../")
sys.path.append("../..")


# Data Ingestion Initialization

## DB Client 

In [2]:
from KETIPreDataIngestion.KETI_setting import influx_setting_KETI as ins
from KETIPreDataIngestion.data_influx import influx_Client_v2 as influx_Client

DBClient = influx_Client.influxClient(ins.CLUSTLocalInflux)
db_list = DBClient.get_DBList()

In [3]:
db_list

['bio_covid_vaccinations',
 'traffic_seoul_subway',
 '_monitoring',
 'city_exhibition_entrance_status',
 'energy_electricity_transaction',
 'energy_water_discharge',
 'energy_wind_power',
 '_tasks',
 'energy_fire_power',
 'farm_strawberry_gunwi',
 'energy_electronic_power',
 'farm_strawberry_jinan',
 'energy_solar',
 'traffic_expressway',
 'farm_strawberry_knu',
 'traffic_public_parking',
 'farm_strawberry_awon',
 'traffic_seoul_bus',
 'traffic_seoul_bike',
 'bio_covid_infected_world',
 'bio_covid_infected_korea',
 'culture_subway']

## DB MS Frequency

In [4]:
db_name ='bio_covid_infected_world'
ms_name = DBClient.measurement_list(db_name)[0]
ms_name

'argentina'

In [5]:
freq = DBClient.get_freq(db_name, ms_name)
freq
print(type(freq))

<class 'dict'>


## DB and Measurement name setting

In [6]:
db_name  = 'bio_covid_infected_world'

In [7]:
ms_list = DBClient.measurement_list(db_name)
ms_name = ms_list[0]
print(ms_name)

argentina


# Data Ingestion Methods

## pure data ingestion

### Ingest Full Data

In [8]:
print(db_name, ms_name)

bio_covid_infected_world argentina


In [9]:
feature_list = DBClient.get_fieldList(db_name, ms_name)
number = "100000"
data = DBClient.get_datafront_by_num(number, db_name, ms_name)
data.head()

,confirmed,deaths
_time,,
2020-01-22 00:00:00+00:00,0,0
2020-01-23 00:00:00+00:00,0,0
2020-01-24 00:00:00+00:00,0,0
2020-01-25 00:00:00+00:00,0,0
2020-01-26 00:00:00+00:00,0,0


In [10]:
first = DBClient.get_first_time(db_name, ms_name)
last = DBClient.get_last_time(db_name, ms_name)
print(first, last)
print(type(last))

2020-01-22T00:00:00Z 2021-11-07T00:00:00Z
<class 'str'>


In [11]:
full_data = DBClient.get_data(db_name, ms_name)

In [12]:
query_end_time = last
query_start_time = first
bind_params = {'end_time':query_end_time, 'start_time': query_start_time}
time_data = DBClient.get_data_by_time(bind_params, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           confirmed  deaths
_time                                       
2020-01-22 00:00:00+00:00          0       0
2020-01-23 00:00:00+00:00          0       0
2020-01-24 00:00:00+00:00          0       0
2020-01-25 00:00:00+00:00          0       0
2020-01-26 00:00:00+00:00          0       0
                           confirmed  deaths
_time                                       
2021-11-02 00:00:00+00:00    5291285  116010
2021-11-03 00:00:00+00:00    5292549  116029
2021-11-04 00:00:00+00:00    5293989  116055
2021-11-05 00:00:00+00:00    5295260  116083
2021-11-06 00:00:00+00:00    5296188  116091


In [13]:
number="20"
time_data = DBClient.get_datafront_by_num(number, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           confirmed  deaths
_time                                       
2020-01-22 00:00:00+00:00          0       0
2020-01-23 00:00:00+00:00          0       0
2020-01-24 00:00:00+00:00          0       0
2020-01-25 00:00:00+00:00          0       0
2020-01-26 00:00:00+00:00          0       0
                           confirmed  deaths
_time                                       
2020-02-06 00:00:00+00:00          0       0
2020-02-07 00:00:00+00:00          0       0
2020-02-08 00:00:00+00:00          0       0
2020-02-09 00:00:00+00:00          0       0
2020-02-10 00:00:00+00:00          0       0


In [14]:
days = 7 
bind_params = {'start_time':query_start_time , 'end_time': query_end_time,"days":str(days)+"d"}
day_data = DBClient.get_data_by_days(bind_params, db_name, ms_name)
print(day_data.head(), day_data.tail())

                           confirmed  deaths
_time                                       
2021-10-31 00:00:00+00:00    5288807  115950
2021-11-01 00:00:00+00:00    5289945  115989
2021-11-02 00:00:00+00:00    5291285  116010
2021-11-03 00:00:00+00:00    5292549  116029
2021-11-04 00:00:00+00:00    5293989  116055                            confirmed  deaths
_time                                       
2021-11-03 00:00:00+00:00    5292549  116029
2021-11-04 00:00:00+00:00    5293989  116055
2021-11-05 00:00:00+00:00    5295260  116083
2021-11-06 00:00:00+00:00    5296188  116091
2021-11-07 00:00:00+00:00    5296781  116104


In [15]:
tag_list = DBClient.get_tagList(db_name, ms_name)
print(db_name)
print(ms_name)

bio_covid_infected_world
argentina


In [16]:
tag_list

['_start', '_stop']

In [17]:
tag_key = 'company'
tag_value='컴캐스트'

df = DBClient.get_TagGroupData(db_name, ms_name, tag_key, tag_value)

In [18]:
tag_distinct_value = DBClient.get_TagValue(db_name, ms_name, tag_key)
print(type(tag_distinct_value))

[<FluxTable: 3 columns, 0 records>]
<class 'list'>


In [19]:
tag_distinct_value

[<FluxTable: 3 columns, 0 records>]

In [20]:
BASE_DIR = os.getcwd()
df_file = "/home/leezy/CLUST_KETI/KETIPreDataIngestion/day_wise.csv"
input_file = os.path.join(BASE_DIR, df_file)
df = pd.read_csv(df_file, parse_dates=True, index_col ='Date')

In [21]:
bk_name = "bk_write_test"
ms_name = "ms_write_test"

In [22]:
DBClient.write_db(bk_name, ms_name, df)


========== create bucket ==========
========== write success ==========
